In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import warnings
warnings.simplefilter(action="ignore")
import time
import pandas as pd

In [22]:
#1. 맥도날드는 페이지를 넘기는 방법을 사용, 행정구역별로 매장을 조회하여 매장 정보를 저장
df = pd.read_csv("data/국토교통부_전국 법정동_20240802.csv")
df = df.loc[df['삭제일자'].isnull(), ['시도명','시군구명']]
df = df.dropna(subset=['시군구명'])
df = df.drop_duplicates().reset_index(drop=True)
df.to_csv('data/전국행정구역.csv', index=False, encoding='utf-8-sig')

In [42]:
#2. 행정구역을 리스트로 불러옴
location = pd.read_csv("data/전국행정구역.csv")
location_list = location['시군구명'].tolist()

In [43]:
#3. 맥드라이브 매장정보 수집
driver = webdriver.Chrome()

url = "https://www.mcdonalds.co.kr/kor/store/list.do"
driver.get(url)
time.sleep(1)
driver.find_element(By.CSS_SELECTOR,"#searchForm > div > div > div > span:nth-child(2) > label").click()
time.sleep(1)

In [44]:
mcd = []
for i in location_list:
    print(i)
    try:
        driver.find_element(By.CSS_SELECTOR,"#searchWord").clear()
    except:
        pass
    driver.find_element(By.CSS_SELECTOR, "#searchWord").send_keys(i)
    time.sleep(1)
    driver.find_element(By.CSS_SELECTOR,"#searchForm > div > fieldset > div > button").click()
    time.sleep(1)
   
    dt_list=driver.find_elements(By.CSS_SELECTOR, "#container > div.content > div.contArea > div > div > div.mcStore > table > tbody > tr")
   
    for data in dt_list:
        address = data.find_element(By.CSS_SELECTOR, "td.tdName > dl > dd").text
        tmp = data.find_element(By.CSS_SELECTOR, "td.tdName > dl > dt > strong > a")
        lat,lng = tmp.get_attribute("href")[19:-2].split(",")
        #원문이 javascript:moveMap(37.5667729,126.9794809);
        #형식이기 때문에 앞에 구문을 지워주고, ","를 기준으로 split해서 저장.
        title = tmp.text
        mcd_labels = ""
        for i in range(len(data.find_elements(By.CSS_SELECTOR, "td.tdService > div > span"))):
            mcd_labels+=data.find_elements(By.CSS_SELECTOR, "td.tdService > div > span")[i].text

        if("맥드라이브" in mcd_labels):
            mcd.append({
                "name":title,
                "address":address,
                "lat":lat,
                "lng":lng
            })

종로구
서울 중구
용산구
성동구
광진구
동대문구
중랑구
성북구
강북구
도봉구
노원구
은평구
서대문구
마포구
양천구
강서구
구로구
금천구
영등포구
동작구
관악구
서초구
강남구
송파구
강동구
부산 중구
부산 서구
부산 동구
영도구
부산 진구
동래구
부산 남구
부산 북구
해운대구
사하구
금정구
강서구
연제구
수영구
사상구
기장군
대구 중구
대구 동구
대구 서구
대구 남구
대구 북구
수성구
달서구
달성군
군위군
인천 중구
인천 동구
미추홀구
연수구
남동구
부평구
계양구
인천 서구
강화군
옹진군
광주 동구
광주 서구
광주 남구
광주 북구
광산구
대전 동구
대전 중구
대전 서구
유성구
대덕구
울산 중구
울산 남구
울산 동구
울산 북구
울주군
세종시
수원시 장안구
수원시 권선구
수원시 팔달구
수원시 영통구
성남시 수정구
성남시 중원구
성남시 분당구
의정부시
안양시
안양시 만안구
안양시 동안구
부천시 원미구
부천시 소사구
부천시 오정구
광명시
평택시
동두천시
안산시
안산시 상록구
안산시 단원구
고양시
고양시 덕양구
고양시 일산동구
고양시 일산서구
과천시
구리시
남양주시
오산시
시흥시
군포시
의왕시
하남시
용인시
용인시 처인구
용인시 기흥구
용인시 수지구
파주시
이천시
안성시
김포시
화성시
광주시
양주시
포천시
여주시
연천군
가평군
양평군
청주시
청주시 상당구
청주시 서원구
청주시 흥덕구
청주시 청원구
충주시
제천시
보은군
옥천군
영동군
증평군
진천군
괴산군
음성군
단양군
천안시
천안시 동남구
천안시 서북구
공주시
보령시
아산시
서산시
논산시
계룡시
당진시
금산군
부여군
서천군
청양군
홍성군
예산군
태안군
목포시
여수시
순천시
나주시
광양시
담양군
곡성군
구례군
고흥군
보성군
화순군
장흥군
강진군
해남군
영암군
무안군
함평군
영광군
장성군
완도군
진도군
신안군
포항시
포항시 남구
포항시 북구
경주시
김천시
안동시
구미시
영주시
영천시
상주시
문경시
경산시
의성군
청송군
영양군
영덕군
청도군
고령군
성주군
칠곡군
예천군
봉화군
울진군
울릉군
창원시
창원시 의창구
창원시 성산구


In [49]:
#4. 현행 주소 체계와 아직 반영 안된 부분 (부천시 구)을 고려하여 수집 후 중복제거
df_mcd=pd.DataFrame(mcd)
df_mcd= df_mcd.drop_duplicates().reset_index(drop=True)

In [50]:
df_mcd.to_csv('data/맥도날드_매장정보.csv', index=False, encoding='utf-8-sig')